In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [34]:
def getSplitPairDataFrame(true_sentence_list,
                          candidate_sentence_list,
                          candidate_label_true_false,
                          true_label_list,
                          candidate_label_list):
    return pd.DataFrame({'true_sentence':true_sentence_list,
                         'candidate_sentence':candidate_sentence_list,
                         'pair_result':candidate_label_true_false,
                         'true_label':true_label_list,
                         'candidate_label':candidate_label_list})
    
def getOutput(mat, index, top):
    return mat[index, :top].ravel()

def getSortedByProbability(original_shape, candidate_mat, probability_mat):
    sorted_candidate_mat = np.zeros((original_shape)).astype('int')
    sorted_probability_mat = np.zeros((original_shape))
    for i in range(original_shape[0]):
        sorted_index = probability_mat[i].argsort()[::-1]
        sorted_candidate_mat[i] = candidate_mat[i, sorted_index]
        sorted_probability_mat[i] = probability_mat[i, sorted_index] 
    return (sorted_candidate_mat, sorted_probability_mat)

In [42]:
%%time
k = 100
split_pair_path = "../data/sep_9_lemonbear_triplet_1epoch/pair_siamese_bert_base_cased_v7.0_triplet_epoch1_k_100.txt"


split_data = pd.read_csv(split_pair_path, sep='\t')

CPU times: user 4.3 s, sys: 228 ms, total: 4.53 s
Wall time: 3.61 s


In [43]:
%%time
total_len = len(split_data)
unique_len = int(total_len/k)
original_shape = (unique_len, k)
true_sentence = np.array(split_data.true_sentence).reshape(original_shape)
candidate_sentence = np.array(split_data.candidate_sentence).reshape(original_shape)
pair_result = np.array(split_data.pair_result).reshape(original_shape)
true_label = np.array(split_data.true_label).reshape(original_shape)
candidate_label = np.array(split_data.candidate_label).reshape(original_shape)

CPU times: user 199 ms, sys: 6.86 ms, total: 206 ms
Wall time: 148 ms


In [45]:
top = 50
np.mean(np.max(pair_result[:,:top], axis=1))

0.4701801901392881

In [38]:
probability_file_path = "/home/weizhuozhang/Desktop/begin_again_2nd_iter_k_50_623w_3epoch/test_results.tsv"


probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]
probability_mat = np.array(probability_file).reshape(original_shape)

In [39]:
%%time
# Sort according to probability, descending
sorted_pair_label, sorted_probability_mat = \
        getSortedByProbability(original_shape,
                               pair_result,
                               probability_mat)

CPU times: user 57.9 ms, sys: 3.41 ms, total: 61.3 ms
Wall time: 225 ms


In [41]:
top = 1
np.mean(np.max(sorted_pair_label[:,:top], axis=1))

0.7201270893769858

In [9]:
np.max(sorted_pair_label[:,:20])

1

In [11]:
# np.mean(np.max(sorted_pair_label[:,:20], axis=1))
np.mean(sorted_pair_label[:,0])

0.30798249585030935

In [13]:
%%time
k_top = 20
# Store the positive pair(Recall pair) which contains true label in "top" candidates
# output_positive_pair_file_path = "../data/split_600w_2epoch/second_positive_pair_result_k_top_{0}_k_{1}_of_k_1000.csv".format(k_top, k)
output_positive_pair_file_path = "../data/42recall_lemonbear_embedding/positive_pair_dev_600w_2epoch_top_20_from_k_top_100.txt"

positive_index = (np.max(sorted_pair_label[:,:k_top], axis=1) == 1)
output_true_sentence = getOutput(true_sentence[:,:k_top], positive_index, k_top)
output_candidate_sentence =  getOutput(candidate_sentence[:,:k_top], positive_index, k_top)
output_pair_result = getOutput(pair_result[:,:k_top], positive_index, k_top)
output_true_label = getOutput(true_label[:,:k_top], positive_index, k_top)
output_candidate_label = getOutput(candidate_label[:,:k_top], positive_index, k_top)
positive_pair_df = getSplitPairDataFrame(output_true_sentence,
                                         output_candidate_sentence,
                                         output_pair_result,
                                         output_true_label,
                                         output_candidate_label)                         
positive_pair_df.to_csv(output_positive_pair_file_path, sep='\t', index=False) 

CPU times: user 2.13 s, sys: 58.6 ms, total: 2.19 s
Wall time: 1.48 s
